In [18]:
import pandas as pd
import xarray as xr
import seaborn as sns
import numpy as np
import os, sys
from dscim.utils.functions import calculate_constant_equivalent_discount_rate

In [13]:
discount_types = ["euler_ramsey", "constant"]
menu_options = ["adding_up", "risk_aversion"]
pulse_years = [2020]
# pulse_years = range(2020, 2081, 10)
weitzman_values = [x / 10.0 for x in range(1, 11, 1)] + [0.25, 0.01, 0.001, 0.0001]
sectors = ["mortality_vsl_popavg", "mortality_vsl_ir", "mortality"]
# sectors = ["CAMEL", "agriculture", "mortality", "energy", "labor"]
AR_list = [6]
eta_rhos = { 
    2.0:0.0,
    1.016010255 : 9.149608e-05,
    1.244459066 : 0.00197263997,
    1.421158116 : 0.00461878399,
    1.567899395 : 0.00770271076
}

CONST_DISC_RATES = [0.01, 0.015, 0.02, 0.025, 0.03, 0.05]
PULSE_YEAR = 2020

combos = [(s, p, d, m, ar, eta) 
          for p in pulse_years
          for s in sectors  
          for d in discount_types 
          for m in menu_options
          for ar in AR_list
          for eta in eta_rhos.keys()]

In [15]:
n_combos = len(combos)

while n_combos > 0: 
    combo = combos[0]
    print("=========================================================")
    print(combo)
    sector = combo[0]
    pulse_year = combo[1]
    disc = combo[2]
    recipe = combo[3]
    AR = combo[4]
    eta = combo[5]
    rho = eta_rhos[eta]

    save_path= f'/mnt/CIL_integration/menu_results_AR{AR}_epa/{sector}/{pulse_year}/unmasked/'
    
    
    md = xr.open_dataset(f"{save_path}/{recipe}_{disc}_eta{eta}_rho{rho}_marginal_damages.nc4").marginal_damages.sel(year=slice(None, 2100))

    if disc == "constant":
        # sccs = (md * np.exp(-md.discrate * (md.year - 2020))).sel(year=slice(None, 2100)).sum('year').rename('scc')
        
        discrate_damages = [
                md * np.exp(-r * (md.year - PULSE_YEAR))
                for r in CONST_DISC_RATES
            ]

        pvd_damages = xr.concat(
            discrate_damages,
            dim=pd.Index(CONST_DISC_RATES, name="discrate"),
        )
        
        scc = pvd_damages.sum('year').rename('scc')
        
    else:
        # get pre2100 SCC
        df = xr.open_dataset(f"{save_path}/{recipe}_{disc}_eta{eta}_rho{rho}_discount_factors.nc4").discount_factors.sel(year=slice(None, 2100))
        scc = (df * md).sum('year').rename('scc')
    
    
    scc.to_netcdf(
            f"{save_path}/{recipe}_{disc}_eta{eta}_rho{rho}_pre2100_scc.nc4")

    combos.remove(combo)
    n_combos = len(combos)
    print(f"remaining combos: {n_combos}")


('mortality_vsl_ir', 2020, 'euler_ramsey', 'risk_aversion', 6, 2.0)
remaining combos: 34
('mortality_vsl_ir', 2020, 'euler_ramsey', 'risk_aversion', 6, 1.016010255)
remaining combos: 33
('mortality_vsl_ir', 2020, 'euler_ramsey', 'risk_aversion', 6, 1.244459066)
remaining combos: 32
('mortality_vsl_ir', 2020, 'euler_ramsey', 'risk_aversion', 6, 1.421158116)
remaining combos: 31
('mortality_vsl_ir', 2020, 'euler_ramsey', 'risk_aversion', 6, 1.567899395)
remaining combos: 30
('mortality_vsl_ir', 2020, 'constant', 'adding_up', 6, 2.0)
remaining combos: 29
('mortality_vsl_ir', 2020, 'constant', 'adding_up', 6, 1.016010255)
remaining combos: 28
('mortality_vsl_ir', 2020, 'constant', 'adding_up', 6, 1.244459066)
remaining combos: 27
('mortality_vsl_ir', 2020, 'constant', 'adding_up', 6, 1.421158116)
remaining combos: 26
('mortality_vsl_ir', 2020, 'constant', 'adding_up', 6, 1.567899395)
remaining combos: 25
('mortality_vsl_ir', 2020, 'constant', 'risk_aversion', 6, 2.0)
remaining combos: 24
(

In [26]:
# RFF
discount_types = ["euler_ramsey", "constant"]
menu_options = ["adding_up", "risk_aversion"]
pulse_years = [2020]
# pulse_years = range(2020, 2081, 10)
weitzman_values = [x / 10.0 for x in range(1, 11, 1)] + [0.25, 0.01, 0.001, 0.0001]
sectors = ["mortality_vsl_popavg", "mortality_vsl_ir", "mortality"]
# sectors = ["CAMEL", "agriculture", "mortality", "energy", "labor"]
AR_list = [6]
eta_rhos = { 
    2.0:0.0,
    1.016010255 : 9.149608e-05,
    1.244459066 : 0.00197263997,
    1.421158116 : 0.00461878399,
    1.567899395 : 0.00770271076
}

CONST_DISC_RATES = [0.01, 0.015, 0.02, 0.025, 0.03, 0.05]
PULSE_YEAR = 2020

combos = [(s, p, d, m, ar, eta) 
          for p in pulse_years
          for s in sectors  
          for d in discount_types 
          for m in menu_options
          for ar in AR_list
          for eta in eta_rhos.keys()]

n_combos = len(combos)

while n_combos > 0: 
    combo = combos[0]
    print("=========================================================")
    print(combo)
    sector = combo[0]
    pulse_year = combo[1]
    disc = combo[2]
    recipe = combo[3]
    AR = combo[4]
    eta = combo[5]
    rho = eta_rhos[eta]

    save_path= f'/mnt/CIL_integration/rff2/{sector}/{pulse_year}/unmasked_None/'
    
    
    md = xr.open_zarr(f"{save_path}/{recipe}_{disc}_eta{eta}_rho{rho}_uncollapsed_marginal_damages.zarr").marginal_damages.sel(year=slice(None, 2100))

    if disc == "constant":
        # sccs = (md * np.exp(-md.discrate * (md.year - 2020))).sel(year=slice(None, 2100)).sum('year').rename('scc')
        
        discrate_damages = [
                md * np.exp(-r * (md.year - PULSE_YEAR))
                for r in CONST_DISC_RATES
            ]

        pvd_damages = xr.concat(
            discrate_damages,
            dim=pd.Index(CONST_DISC_RATES, name="discrate"),
        )
        
        scc = pvd_damages.sum('year').rename('scc')
        
    else:
        # get pre2100 SCC
        df = xr.open_zarr(f"{save_path}/{recipe}_{disc}_eta{eta}_rho{rho}_uncollapsed_discount_factors.zarr").discount_factor.sel(year=slice(None, 2100))
        scc = (df * md).sum('year').rename('scc')
    
    
    scc.to_netcdf(
            f"{save_path}/{recipe}_{disc}_eta{eta}_rho{rho}_uncollapsed_pre2100_scc.nc4")

    combos.remove(combo)
    n_combos = len(combos)
    print(f"remaining combos: {n_combos}")


('mortality_vsl_popavg', 2020, 'euler_ramsey', 'adding_up', 6, 2.0)
remaining combos: 59
('mortality_vsl_popavg', 2020, 'euler_ramsey', 'adding_up', 6, 1.016010255)
remaining combos: 58
('mortality_vsl_popavg', 2020, 'euler_ramsey', 'adding_up', 6, 1.244459066)
remaining combos: 57
('mortality_vsl_popavg', 2020, 'euler_ramsey', 'adding_up', 6, 1.421158116)
remaining combos: 56
('mortality_vsl_popavg', 2020, 'euler_ramsey', 'adding_up', 6, 1.567899395)
remaining combos: 55
('mortality_vsl_popavg', 2020, 'euler_ramsey', 'risk_aversion', 6, 2.0)
remaining combos: 54
('mortality_vsl_popavg', 2020, 'euler_ramsey', 'risk_aversion', 6, 1.016010255)
remaining combos: 53
('mortality_vsl_popavg', 2020, 'euler_ramsey', 'risk_aversion', 6, 1.244459066)
remaining combos: 52
('mortality_vsl_popavg', 2020, 'euler_ramsey', 'risk_aversion', 6, 1.421158116)
remaining combos: 51
('mortality_vsl_popavg', 2020, 'euler_ramsey', 'risk_aversion', 6, 1.567899395)
remaining combos: 50
('mortality_vsl_popavg', 2

In [27]:
xr.open_dataset(f"{save_path}/{recipe}_{disc}_eta{eta}_rho{rho}_uncollapsed_pre2100_scc.nc4")

<xarray.Dataset>
Dimensions:             (discount_type: 1, gas: 3, runid: 10000, weitzman_parameter: 14, discrate: 6)
Coordinates:
  * discount_type       (discount_type) object 'constant'
  * gas                 (gas) object 'CO2_Fossil' 'CH4' 'N2O'
  * runid               (runid) int64 1 2 3 4 5 6 ... 9996 9997 9998 9999 10000
  * weitzman_parameter  (weitzman_parameter) object '0.1' '0.2' ... '0.0001'
  * discrate            (discrate) float64 0.01 0.015 0.02 0.025 0.03 0.05
Data variables:
    scc                 (discrate, weitzman_parameter, discount_type, runid, gas) float64 ...

In [30]:
xr.open_dataset(f"{save_path}/risk_aversion_euler_ramsey_eta{eta}_rho{rho}_uncollapsed_pre2100_scc.nc4")

<xarray.Dataset>
Dimensions:             (discount_type: 1, gas: 3, region: 1, runid: 10000, weitzman_parameter: 14)
Coordinates:
  * discount_type       (discount_type) object 'euler_ramsey'
  * gas                 (gas) object 'CO2_Fossil' 'CH4' 'N2O'
  * region              (region) object 'world'
  * runid               (runid) int64 1 2 3 4 5 6 ... 9996 9997 9998 9999 10000
  * weitzman_parameter  (weitzman_parameter) object '0.1' '0.2' ... '0.0001'
Data variables:
    scc                 (weitzman_parameter, discount_type, runid, gas, region) float64 ...